#### Reading data from csv

In [39]:
import pandas as pd
import numpy as np
df = pd.read_csv('UNSPSCdataset.csv',encoding='mac_roman',low_memory=False)
test_df = pd.read_csv('UNSPSCtestDataSet.csv',encoding='mac_roman',low_memory=False)
# y and X are labels and features for the training set respectively
df = df.drop_duplicates(['MaterialDescription'])
y = df['UNSPSC_Final']
X = df['MaterialDescription']
X.head()

0      AIRTEL BILLS 22aug TO 23 AUG    2012
1     AIRTEL MOBILE BILLS 23nov O 22 dec 12
2                 aluminum fabrication work
4               civil & plumbing work @ BMT
5             ELECTRICAL WK 09ELR LAB @ MHB
Name: MaterialDescription, dtype: object

In [40]:
def description_to_words(sentence):
    # Keeping only text
    import re
    from nltk.corpus import stopwords
    text = re.sub("[^a-zA-Z0-9]"," ",sentence)
    # Convert words to lower case and split them 
    text = text.lower().split(" ")
    # optionally remove stopwords
    stops = set(stopwords.words('english'))
    meaningful_words = [w for w in text if not w in stops]
    # Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

def sentences_split(field):
    from nltk.corpus import stopwords
    # Tokenizing each sentence from the paragraph 
    #raw_sentences = tokenizer.tokenize(word.strip())
    
    # Sentence list 
    sentences = []
    if len(field) > 0:
        sentences.append(description_to_words(field))
    return sentences

def list_of_words(series):
    mainsentences = []
    for i in series:
       mainsentences += sentences_split(i)
    x = get_features(mainsentences)
    return x
    

# BAG OF WORDS MODEL 
def get_features(series):
    from sklearn.feature_extraction.text import CountVectorizer

    # Initialize the "CountVectorizer" object, which is scikit-learn's
    # bag of words tool.  
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 5000) 
    train_data_features = vectorizer.fit_transform(series)

    # Numpy arrays are easy to work with, so convert the result to an 
    # array
    train_data_features = train_data_features.toarray()
    return train_data_features


### Multilayer perceptron model in Tensorflow

In [41]:
### Divdindg total data into batches

def batches(k_batches, features, labels): 
    assert len(features) == len(labels)
    batches_total = []
    total_size = len(features)
    for start_i in range(0,total_size, k_batches):
        end_i = start_i + k_batches
        batch = [features[start_i : end_i], labels[start_i : end_i]]
        batches_total.append(batch)
    return batches_total

In [16]:
len(X)

34591

In [8]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
        C:\Users\sunka\AppData\Roaming\nltk_data...
      Unzipping corpora\stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [8]:
unique_train_labels = y.unique()
#print(len(unique_train_labels)) 1161

#print(len(test_labels.unique())) 663

bigdata = pd.concat([y,test_labels], ignore_index=True)
print(len(bigdata.unique()))

1161


In [62]:
train_features = list_of_words(X[:int(len(X)-len(X)/3)])
train_labels = y[:int(len(X)-len(X)/3)]
val_features = list_of_words(X[int(len(X)-len(X)/3):])
val_labels =y[int(len(X)-len(X)/3):]
test_features = list_of_words(test_df['MaterialDescription'])
test_labels = test_df['UNSPSC_Final']
print(train_features.shape)
print(val_features.shape)
print(test_features.shape)

(23060, 5000)
(11531, 5000)
(49657, 5000)


In [63]:
import tensorflow as tf
from sklearn import preprocessing
train_le = preprocessing.LabelEncoder()
test_le = preprocessing.LabelEncoder()
val_le = preprocessing.LabelEncoder()
with tf.Session() as sessold:
    train_labels = tf.one_hot(train_le.fit_transform(train_labels), n_classes).eval(session = sessold)
    test_labels = tf.one_hot(test_le.fit_transform(test_labels), n_classes).eval(session = sessold)
    val_labels = tf.one_hot(val_le.fit_transform(val_labels), n_classes).eval(session = sessold)
    print(test_labels.shape)
    print(train_labels.shape)
    print(val_labels.shape)


(49657, 1161)
(23060, 1161)
(11531, 1161)


In [64]:
import time
start = time.time()
n_inputs = 5000
n_classes = 1161


features = tf.placeholder(tf.float32, [None, n_inputs])
labels = tf.placeholder(tf.int32, [None,n_classes])

h_layer_1 = 3081
h_layer_2 = 2500
h_layer_3 = 2000

weights = {
'hidden_weights1' : tf.Variable(tf.truncated_normal([n_inputs, h_layer_1])),
'hidden_weights2' : tf.Variable(tf.truncated_normal([h_layer_1, h_layer_2])),
'hidden_weights3' : tf.Variable(tf.truncated_normal([h_layer_2, h_layer_3])),
'out_weights' : tf.Variable(tf.truncated_normal([h_layer_3, n_classes]))
}

bias = {
'hidden_bias1' : tf.Variable(tf.truncated_normal([h_layer_1])),
'hidden_bias2' : tf.Variable(tf.truncated_normal([h_layer_2])),
'hidden_bias3' : tf.Variable(tf.truncated_normal([h_layer_3])),
'out_bias' : tf.Variable(tf.truncated_normal([n_classes]))
}
keep_prob = tf.placeholder(tf.float32)

### Hidden layer 1 
hidden_output1 = tf.add(tf.matmul(features, weights['hidden_weights1']),bias['hidden_bias1'])
hidden_relu1 = tf.nn.relu(hidden_output1)
hidden_out1 = tf.nn.dropout(hidden_relu1, keep_prob)

### Hidden layer 2
hidden_output2 = tf.add(tf.matmul(hidden_out1, weights['hidden_weights2']),bias['hidden_bias2'])
hidden_relu2 = tf.nn.relu(hidden_output2)
hidden_out2 = tf.nn.dropout(hidden_relu2, keep_prob)

### Hidden layer 3
hidden_output3 = tf.add(tf.matmul(hidden_out2, weights['hidden_weights3']),bias['hidden_bias3'])
hidden_relu3 = tf.nn.relu(hidden_output3)
hidden_out3 = tf.nn.dropout(hidden_relu3, keep_prob)

### Output layer 
logits = tf.add(tf.matmul(hidden_out3, weights['out_weights']),bias['out_bias'])
learn_rate = 0.5 # 0.5 + 200 with a dropout in output layer + 0.7prob for training set= 99.6%

# The file path to save the data
save_file = './train_model.ckpt'
saver = tf.train.Saver() # Class to save the model


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learn_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

batchSize =  256 

epochs = 20 #  200epoch+.5lr = 99.6
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    total_batches = batches(batchSize, train_features, train_labels)   

    for epoch in range(epochs): 
        for batch_features, batch_labels in total_batches: 
            train_data = {features: batch_features, labels : batch_labels, keep_prob : 0.5}
            
            sess.run(optimizer, feed_dict = train_data)
        # Print status for every 100 epochs
        
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: val_features,
                    labels: val_labels,
                    keep_prob : 0.5})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))
    Accuracy = sess.run(accuracy, feed_dict={features : test_features, labels :test_labels, keep_prob : 1.0})
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')
    
    prediction = tf.argmax(logits,1)
    output_array = le.inverse_transform(prediction.eval(feed_dict={features : test_features, keep_prob: 1.0}))
    prediction = np.reshape(prediction, (test_features.shape[0],1))
    np.savetxt("prediction.csv", prediction, delimiter=",")

print("Accuracy value is {}".format(Accuracy))
end = time.time()
print("total time ", end-start)

Epoch 0   - Validation Accuracy: 0.034862544387578964
Epoch 10  - Validation Accuracy: 0.03486254811286926


ResourceExhaustedError: OOM when allocating tensor with shape[49657,3081]
	 [[Node: MatMul_28 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_Placeholder_21_0_0/_17, Variable_56/read)]]

Caused by op 'MatMul_28', defined at:
  File "C:\Anaconda\envs\gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda\envs\gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\envs\gpu\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda\envs\gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda\envs\gpu\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda\envs\gpu\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\

#### Restoring the saved model 


In [ ]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(accuracy, feed_dict={features : test_features, labels :test_labels, keep_prob : 1.0})
    
    outputs = sess.run()
print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
